# Twitter Data Wrangling 
MAHMOUD NAGY - MAR 2022

In [1]:
import pandas as pd
import os

In [2]:
cd ../../Twitter_Data

/Users/mnagy99/jupyter/AH/Twitter_Bot_Analysis/Twitter_Data


<br>

<br>

# Other Languages Wrangling

In [3]:
df_merged = pd.read_csv("merged_all_years_zip.csv", compression='zip', lineterminator='\n')

In [25]:
# check the undefined
df_merged[df_merged['lang'] == 'und'].text.value_counts().head(10)

#JusticeForJohnnyDepp                            41688
#JusticeForJohnnyDepp #AmberHeardIsAnAbuser       1554
#AmberHeardIsAnAbuser                             1543
#justiceforjohnnydepp                             1482
#AmberHeardIsALiar                                1314
#JusticeForJohnnyDepp ⚖️                          1303
@realamberheard #JusticeForJohnnyDepp              711
#JusticeforJohnnyDepp                              706
#JusticeForJohnnyDepp \n#AmberHeardIsAnAbuser      535
#JusticeForJohnnyDepp \n#JohnnyDepp                512
Name: text, dtype: int64

In [26]:
df_merged[df_merged['lang'] == 'und'].shape

(173082, 22)

In [27]:
df_threat_others = df_merged[(df_merged['lang'] != 'en') & (df_merged['lang'] != 'und')]
print(df_threat_others.shape)

(143304, 22)


In [28]:
# Add A Translation Column
import mtranslate

def translate(x):
    try:
        translated = mtranslate.translate(x,"en","auto")
    except:
        translated = 'not_translated'
    return translated

In [29]:
import time
t0 = time.time()
df_threat_others['translated'] = df_threat_others.text.apply(lambda x: translate(x))
(time.time() - t0) /3600

<ipython-input-29-d86baf97233e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threat_others['translated'] = df_threat_others.text.apply(lambda x: translate(x))


19.811731179753938

In [30]:
df_threat_others[df_threat_others.translated == 'not_translated'].shape

(10091, 23)

# Create two data frames then join to translate the remaining text

In [31]:
df_translated = df_threat_others[df_threat_others['translated'] != 'not_translated']
df_translated.shape

(133213, 23)

In [32]:
df_not_translated = df_threat_others[df_threat_others['translated'] == 'not_translated']
df_not_translated.shape

(10091, 23)

In [33]:
import time
t0 = time.time()
df_not_translated['translated'] = df_not_translated.text.apply(lambda x: translate(x))
(time.time() - t0) /3600

<ipython-input-33-57fde3d2e6c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_translated['translated'] = df_not_translated.text.apply(lambda x: translate(x))


1.3790832949346967

In [34]:
# join dataframes and sort by date
df_threat_others = pd.concat([df_translated, df_not_translated])
df_threat_others = df_threat_others.sort_values('created_at')
df_threat_others.shape

(143304, 23)

In [35]:
df_not_translated = df_threat_others[df_threat_others['translated'] == 'not_translated']
df_not_translated.shape

(0, 23)

In [36]:
df_threat_others.to_csv("other_languages.csv", index=False)

<br>

# Load Data

In [4]:
df_threat_others = pd.read_csv("other_languages.csv", lineterminator='\n')
df_threat_others.rename(columns={'translated\r': 'translated'}, inplace=True)
print(df_threat_others.shape)
df_threat_others.head(1)

(143304, 24)


,user_description,username,n_followers,text,profile_banner_url,user_location,user_id,user_created_at,profile_image_url,n_friends,...,tweet_id,user_screen_name,date,year,month,dayofmonth,diff,days_diff,translated,hours_diff
0,28. dark and twisty♡ taking it sleazy✌🏻 tv sho...,Caro,3427,Amber Heard anduvo en Chilito?,https://pbs.twimg.com/profile_banners/10562024...,"Santiago, Chile",105620244,2010-01-16 23:16:06,http://pbs.twimg.com/profile_images/1378557216...,3556,...,947652598630404101,iCaaroooh,2018-01-01,2018,Jan,1,2906 days 02:59:54,2906,Amber Heard anduvo en Chilito?\r,2


# Normalize text and Translated Text

In [5]:
df_threat_others['text'] = df_threat_others['text'].str.lower()
df_threat_others['translated'] = df_threat_others['translated'].str.lower()

In [6]:
df_threat_others.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143304 entries, 0 to 143303
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   user_description    127062 non-null  object
 1   username            143288 non-null  object
 2   n_followers         143304 non-null  int64 
 3   text                143304 non-null  object
 4   profile_banner_url  132198 non-null  object
 5   user_location       96680 non-null   object
 6   user_id             143304 non-null  int64 
 7   user_created_at     143304 non-null  object
 8   profile_image_url   143295 non-null  object
 9   n_friends           143304 non-null  int64 
 10  lang                143304 non-null  object
 11  user_url            59928 non-null   object
 12  created_at          143304 non-null  object
 13  n_statuses          143304 non-null  int64 
 14  tweet_id            143304 non-null  int64 
 15  user_screen_name    143304 non-null  object
 16  da

In [11]:
# change dtypes
df_threat_others['user_id'] = df_threat_others['user_id'].astype('str')
df_threat_others['tweet_id'] = df_threat_others['tweet_id'].astype('str')

# use format="%Y-%m-%d %H:%M:%S+00:00 to remove the +00:00
df_threat_others["created_at"] = pd.to_datetime(df_threat_others["created_at"])
df_threat_others["user_created_at"] = pd.to_datetime(df_threat_others["user_created_at"])

In [12]:
# Adding year, month, dayofmonth and a diff columns
df_threat_others["date"] = df_threat_others["created_at"].dt.date
df_threat_others["year"] = df_threat_others["created_at"].dt.year
df_threat_others["month"] = df_threat_others["created_at"].dt.strftime("%b")
df_threat_others["dayofmonth"] = df_threat_others["created_at"].dt.day
# df_merged["hour"] = df_merged["created_at"].dt.hour
df_threat_others["diff"] = df_threat_others["created_at"] - df_threat_others["user_created_at"]
df_threat_others["days_diff"] = df_threat_others["diff"].dt.days
df_threat_others["hours_diff"] = df_threat_others["diff"].dt.components['hours']

In [14]:
df_threat_others.head(1)

,user_description,username,n_followers,text,profile_banner_url,user_location,user_id,user_created_at,profile_image_url,n_friends,...,tweet_id,user_screen_name,date,year,month,dayofmonth,diff,days_diff,translated,hours_diff
0,28. dark and twisty♡ taking it sleazy✌🏻 tv sho...,Caro,3427,amber heard anduvo en chilito?,https://pbs.twimg.com/profile_banners/10562024...,"Santiago, Chile",105620244,2010-01-16 23:16:06,http://pbs.twimg.com/profile_images/1378557216...,3556,...,947652598630404101,iCaaroooh,2018-01-01,2018,Jan,1,2906 days 02:59:54,2906,amber heard anduvo en chilito?\r,2


In [15]:
df_threat_others.to_csv("other_languages.csv", index=False)